# Dataset creation for classification

In [1]:
#!pip install tensorflow
#!pip install seaborn

In [2]:
import pandas as pd
import nltk
from utils_processor.processor import Processor
import logging
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from gensim.models import Word2Vec
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score
from typing import Tuple, List, Any
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout, LSTM, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np
from typing import Tuple, Any
from tensorflow.keras.layers import (
        Input, Bidirectional, LayerNormalization, SpatialDropout1D,
        BatchNormalization, Add, MultiHeadAttention, 
        Concatenate, GlobalAveragePooling1D
    )
from tensorflow.keras.models import Model

In [3]:
processor_ = Processor()

In [4]:
import os
import re

# Directorio donde están los archivos .txt
data_dir = 'data/books/'

# Lista para almacenar los textos y autores
texts = []
authors = []

# Leer todos los archivos .txt del directorio
for filename in os.listdir(data_dir):
    if filename.endswith('.txt'):

        with open(os.path.join(data_dir, filename), 'r', encoding='utf-8') as f:
            text = f.read()
            texts.append(text)  # Almacenar el texto
            print(f"Procesando archivo: {filename}, lineas: ", len(text))
            
            # Buscar el nombre del autor
            author_match = re.search(r'Author:\s*(.+)', text)
            if author_match:
                author_name = author_match.group(1).strip()
                authors.append(author_name.lower())  # Almacenar el autor
            else:
                authors.append("Autor no encontrado")  # En caso de no encontrarlo
            
            # Mostrar un fragmento del texto (opcional)
            #print(text[:2500])

# Mostrar los autores encontrados
print("Lista de autores en orden:")
for i, author in enumerate(authors):
    print(f"{i+1}. {author}")

# Ahora tienes dos listas: 'texts' con los textos y 'authors' con los autores en el mismo orden
authors

Procesando archivo: ATaleofTwoCities_Dickens.txt, lineas:  776878
Procesando archivo: BleakHouse_Dickens.txt, lineas:  1958792
Procesando archivo: CountofMonteCristo_Dumas.txt, lineas:  2646641
Procesando archivo: CrimeAndPunishment_dostoyevski.txt, lineas:  1154409
Procesando archivo: OliverTwist_Dickens.txt, lineas:  912421
Procesando archivo: TheGambler_dostoyevski.txt, lineas:  350954
Procesando archivo: TheIdiot_dostoyevski.txt, lineas:  1366983
Procesando archivo: TheThreeMusketeers_Dumas.txt, lineas:  1317339
Procesando archivo: TwentyYearsAfter_Dumas.txt, lineas:  1387344
Lista de autores en orden:
1. charles dickens
2. charles dickens
3. alexandre dumas
4. fyodor dostoyevsky
5. charles dickens
6. fyodor dostoyevsky
7. fyodor dostoyevsky
8. alexandre dumas
9. alexandre dumas


['charles dickens',
 'charles dickens',
 'alexandre dumas',
 'fyodor dostoyevsky',
 'charles dickens',
 'fyodor dostoyevsky',
 'fyodor dostoyevsky',
 'alexandre dumas',
 'alexandre dumas']

In [5]:
def process_all_texts(processor = Processor(), texts: list = []):
    """
    Processes a list of texts and logs progress for each one, using the Processor class.
    
    Args:
        processor (Processor): An instance of the Processor class.
        texts (list): A list of text strings to process.
    
    Returns:
        list: A list of processed texts.
    """
    total = len(texts)
    processed_texts = []
    
    for index, text in enumerate(texts):
        processed_text = processor.preprocessing_pipeline_as_chunks(text, index, total)
        processed_texts.append(processed_text)  # Guardamos el texto procesado como lista de tokens
    
    return processed_texts

In [6]:
# Procesar todos los textos con el sistema de logging
processed_texts = process_all_texts(processor_, texts)

In [7]:
text_chunks = []
chunk_authors = []

for i, text_list in enumerate(processed_texts):
    author = authors[i]
    for chunk in text_list:
        text_chunks.append(chunk)  # Agregar cada chunk de texto
        chunk_authors.append(author)  # Agregar el autor correspondiente

# Crear un DataFrame con las listas
df_chunks = pd.DataFrame({
    'text_chunk': text_chunks,
    'author': chunk_authors
})


In [8]:
df_chunks

,text_chunk,author
0,tale num citi tale num citi stori french revol...,charles dickens
1,chapter xv knit chapter xvi still knit chapter...,charles dickens
2,life chapter period best time worst time age w...,charles dickens
3,king larg jaw queen plain face throne england ...,charles dickens
4,london westminst even cocklan ghost laid round...,charles dickens
...,...,...
16890,critic reach project gutenberg goal ensur proj...,alexandre dumas
16891,num contribut project gutenberg literari archi...,alexandre dumas
16892,num num particular import maintain tax exempt ...,alexandre dumas
16893,us offer donat intern donat grate accept make ...,alexandre dumas


1. Prepare the text data

We already have the processed texts stored in a list called processed_texts. Each element in this list represents the chunks of text (after splitting) for a particular book.
Each entry in processed_texts is a list where each element is a chunk of text for that book, processed based on the method we implemented for splitting into chunks of 150 words with a 25-word overlap

2. Prepare the author labels

We have an authors list that stores the corresponding author for each book in processed_texts. Each author appears multiple times if they have multiple books in the dataset. 

3. Create the DataFrame structure

For each processed book (i.e., processed_texts[i]), we know that all the chunks of that book correspond to a specific author. So we can assign the same author to all the chunks in that list.
We will loop over each entry in processed_texts and for each chunk, add it to a DataFrame, along with the corresponding author.

4. Steps to build the DataFrame

* Initialize lists for the DataFrame: We will initialize two lists: one for text chunks and one for authors.
* Iterate over processed_texts: For each entry in processed_texts, we extract the list of chunks and the corresponding author.
* Add chunks and authors to the lists: For each chunk in the list of text chunks, we append it to the "text_chunk" list and the corresponding author to the "author" list.
* Create the DataFrame: Once the lists are filled, we create a pandas DataFrame with two columns: "text_chunk" and "author".

In [9]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming df_chunks is the dataframe with the columns ['text_chunk', 'author']

# Step 1: Split the dataset into 70% training and 30% test
train_df, test_df = train_test_split(df_chunks, test_size=0.30, stratify=df_chunks['author'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, stratify=train_df['author'], random_state=42)


In [10]:
train_df

,text_chunk,author
10344,elbow beg pardon said dodger look air abstract...,charles dickens
2552,pursu say emphat william guppi drop mr guppi a...,charles dickens
13609,secret use know anyth said young woman instinc...,alexandre dumas
3369,look keep secret condescens present visit feel...,charles dickens
9896,jew sooner alon counten resum former express a...,charles dickens
...,...,...
5799,better inform know owner hors shut cri pit cho...,alexandre dumas
8183,began count third excus would say fanci made m...,fyodor dostoyevsky
13251,take away commiss give mademoisell de chemerau...,alexandre dumas
4851,must curios natur island mass rock contain acr...,alexandre dumas


In [11]:
val_df

,text_chunk,author
5881,deceiv play joke excel read ah true said mont ...,alexandre dumas
882,heel head wish wos still say prewar sir let bo...,charles dickens
16471,would choos num atho artagnan said noth silenc...,alexandre dumas
3294,littl earlier morn keep account attend houseke...,charles dickens
13959,shall get back upon lackey hors _pardieu_ anyb...,alexandre dumas
...,...,...
8226,heart long anoth father polya papa fear angri ...,fyodor dostoyevsky
11409,note often grow paler take princ took note fer...,fyodor dostoyevsky
7376,room first floor room whitewash custom prison ...,alexandre dumas
1898,time littl woman ad rub head signific settl ye...,charles dickens


In [12]:
test_df

,text_chunk,author
12339,quit sure reach culmin point happi num day saw...,fyodor dostoyevsky
4185,spain itali mercédè father could join fear liv...,alexandre dumas
15378,found pale fatigu inquir whether ill fact said...,alexandre dumas
9926,empti comfort said mrs corney much inde said b...,charles dickens
6920,crush singl touch word breath yes self thought...,alexandre dumas
...,...,...
2448,poor dear girl found much admir good disposit ...,charles dickens
14473,dispos convers reclin corner carriag num pass ...,alexandre dumas
14223,smile indic knew stori well wish relat recomme...,alexandre dumas
16470,shall begin portho arami drew back disappoint ...,alexandre dumas


In [13]:

def summary_by_author(train_df, validation_df, test_df):
    """
    Generates a summary table showing the number of samples per author for the training, validation, and testing sets.
    
    Args:
        train_df (pd.DataFrame): Training DataFrame.
        validation_df (pd.DataFrame): Validation DataFrame.
        test_df (pd.DataFrame): Testing DataFrame.
        
    Returns:
        pd.DataFrame: A summary DataFrame.
    """
    
    summary_data = {
        'Author': train_df['author'].unique(),
        'Train': train_df['author'].value_counts(),
        'Validation': validation_df['author'].value_counts(),
        'Test': test_df['author'].value_counts()
    }
    
    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.fillna(0)  # Replace NaN with 0 if no samples exist for some authors
    
    return summary_df

In [14]:
summary_by_author(train_df, val_df, test_df)

,Author,Train,Validation,Test
author,,,,
alexandre dumas,charles dickens,4744,527,2260
charles dickens,alexandre dumas,3307,368,1575
fyodor dostoyevsky,fyodor dostoyevsky,2592,288,1234


Feed Forward

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from gensim.models import Word2Vec


In [16]:
def prepare_data(df, processor_, word2vec_model):
    X = []
    print("Procesando textos...")
    
    for text in df['text_chunk']:
        processed_tokens = processor_.preprocessing_pipeline(text)
        X.append(processed_tokens)
    
    try:
        vocab = word2vec_model.wv.key_to_index
    except AttributeError:
        vocab = word2vec_model.key_to_index
    X_indices = []
    for tokens in X:
        indices = []
        for token in tokens:
            indices.append(vocab.get(token, 0))  # 0 para tokens desconocidos
        X_indices.append(indices)
    
    return X_indices

In [17]:

def create_embedding_matrix(train_df, val_df, test_df, word2vec_model: Word2Vec, embedding_size: int= 200):

    X_train = prepare_data(train_df, processor_, word2vec_model=word2vec_model)
    X_val = prepare_data(val_df, processor_, word2vec_model= word2vec_model)
    X_test = prepare_data(test_df, processor_, word2vec_model=word2vec_model)

    max_length = max(
        max(len(x) for x in X_train),
        max(len(x) for x in X_val),
        max(len(x) for x in X_test)
    )
    print(f"Longitud máxima de secuencia: {max_length}")

    X_train_padded = pad_sequences(X_train, maxlen=max_length, padding='post')
    X_val_padded = pad_sequences(X_val, maxlen=max_length, padding='post')
    X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post')

    label_encoder = LabelEncoder()
    label_encoder.fit(train_df['author'])

    y_train = label_encoder.transform(train_df['author'])
    y_val = label_encoder.transform(val_df['author'])
    y_test = label_encoder.transform(test_df['author'])

    try:
        vocab = word2vec_model.wv.key_to_index  
    except AttributeError:
        print("Glove model detected")
        vocab = word2vec_model.key_to_index 

    vocab_size = len(vocab) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_size))
        
    for word, idx in vocab.items():
        try:
            embedding_matrix[idx] = word2vec_model.wv[word]
        except AttributeError:
            embedding_matrix[idx] = word2vec_model[word]

    return embedding_matrix

In [18]:
def RDD1(embedding_matrix: np.ndarray, vocab_size: int, embedding_size: int, max_length: int) -> Sequential:
    """
    Creates a neural network architecture with multiple dense layers and dropout for author classification.
    This model uses a pre-trained embedding layer followed by several dense layers with increasing complexity.

    Architecture details:
    - Non-trainable embedding layer using pre-trained weights
    - Flatten layer to convert 3D tensor to 2D
    - Three dense layers with increasing complexity
    - Dropout layers to prevent overfitting
    - Softmax output layer for classification

    Args:
        embedding_matrix (np.ndarray): Pre-trained embedding matrix for word vectors
        vocab_size (int): Size of the vocabulary (number of unique words)
        embedding_size (int): Dimension of the embedding vectors
        max_length (int): Maximum length of input sequences

    Returns:
        Sequential: Compiled Keras model ready for training
    """
    model = Sequential([
        Embedding(input_dim=vocab_size, 
                 output_dim=embedding_size, 
                 weights=[embedding_matrix], 
                 input_length=max_length, 
                 trainable=False),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')  # 3 classes for authors
    ])
    return model

def RDD2(embedding_matrix: np.ndarray, vocab_size: int, embedding_size: int, max_length: int) -> Sequential:
    """
    Creates an advanced neural network architecture combining LSTM and dense layers for author classification.
    This model is designed to capture both sequential patterns and complex feature interactions.

    Architecture details:
    - Non-trainable embedding layer using pre-trained weights
    - Bidirectional LSTM layer for sequence processing
    - Global max pooling to reduce dimensionality
    - Multiple dense layers with batch normalization
    - Dropout layers for regularization
    - Softmax output layer for classification

    Args:
        embedding_matrix (np.ndarray): Pre-trained embedding matrix for word vectors
        vocab_size (int): Size of the vocabulary (number of unique words)
        embedding_size (int): Dimension of the embedding vectors
        max_length (int): Maximum length of input sequences

    Returns:
        Sequential: Compiled Keras model ready for training
    """
    model = Sequential([
        Embedding(input_dim=vocab_size, 
                 output_dim=embedding_size, 
                 weights=[embedding_matrix], 
                 input_length=max_length, 
                 trainable=False),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.4),
        Dense(256, activation='relu'),
        Dropout(0.4),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')  # 3 classes for authors
    ])
    return model

def RDD3(embedding_matrix: np.ndarray, vocab_size: int, embedding_size: int, max_length: int) -> Model:
    """
    Creates an advanced neural network architecture combining Bidirectional LSTM layers,
    residual connections, and attention mechanisms for sophisticated text processing.
    This model is designed for complex natural language understanding tasks.

    Architecture details:
    - Non-trainable embedding layer using pre-trained weights
    - Multiple Bidirectional LSTM layers with residual connections
    - Multi-head attention layer for capturing long-range dependencies
    - Advanced regularization techniques:
        * Spatial Dropout for embeddings
        * Layer normalization
        * Gradient clipping
        * Multiple dropout layers with varying rates
    - Batch normalization for stable training
    - Skip connections between layers
    
    Key features:
    - Deep architecture with 3 Bidirectional LSTM layers
    - Attention mechanism for better context understanding
    - Residual connections to prevent vanishing gradients
    - Multiple regularization techniques to prevent overfitting
    - Progressive dimension reduction through the network

    Args:
        embedding_matrix (np.ndarray): Pre-trained embedding matrix for word vectors
        vocab_size (int): Size of the vocabulary (number of unique words)
        embedding_size (int): Dimension of the embedding vectors
        max_length (int): Maximum length of input sequences

    Returns:
        Model: Compiled Keras model ready for training
    """

    
    # Input layer
    inputs = Input(shape=(max_length,))
    
    # Embedding layer with spatial dropout
    x = Embedding(input_dim=vocab_size,
                 output_dim=embedding_size,
                 weights=[embedding_matrix],
                 trainable=False)(inputs)
    x = SpatialDropout1D(0.2)(x)
    
    # First Bidirectional LSTM with residual connection
    lstm_out = 256
    main_output = Bidirectional(LSTM(lstm_out, 
                                   return_sequences=True,
                                   dropout=0.2,
                                   recurrent_dropout=0.2))(x)
    main_output = LayerNormalization()(main_output)
    
    # Multi-head attention layer
    attention_output = MultiHeadAttention(
        num_heads=8, 
        key_dim=32
    )(main_output, main_output)
    
    # Add & Normalize (residual connection)
    main_output = Add()([main_output, attention_output])
    main_output = LayerNormalization()(main_output)
    
    # Second Bidirectional LSTM
    lstm_2 = Bidirectional(LSTM(lstm_out//2,
                               return_sequences=True,
                               dropout=0.2,
                               recurrent_dropout=0.2))(main_output)
    lstm_2 = LayerNormalization()(lstm_2)
    
    # Third Bidirectional LSTM
    lstm_3 = Bidirectional(LSTM(lstm_out//4,
                               return_sequences=True,
                               dropout=0.2,
                               recurrent_dropout=0.2))(lstm_2)
    lstm_3 = LayerNormalization()(lstm_3)
    
    # Combine different levels of features
    pooled_outputs = [
        GlobalAveragePooling1D()(main_output),
        GlobalAveragePooling1D()(lstm_2),
        GlobalAveragePooling1D()(lstm_3)
    ]
    concat = Concatenate()(pooled_outputs)
    
    # Dense layers with batch normalization and dropout
    x = Dense(512, activation='relu')(concat)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # Output layer
    outputs = Dense(3, activation='softmax')(x)
    
    # Create model
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

def train_model(model: Sequential, 
                train_data: Tuple[np.ndarray, np.ndarray],
                val_data: Tuple[np.ndarray, np.ndarray],
                batch_size: int = 32,
                epochs: int = 20) -> Any:
    """
    Trains the neural network model with early stopping and learning rate scheduling.
    Implements best practices for model training including early stopping and 
    performance monitoring.

    Training features:
    - Early stopping to prevent overfitting
    - Sparse categorical crossentropy loss for multi-class classification
    - Adam optimizer with default learning rate
    - Batch training with customizable batch size
    - Validation monitoring during training

    Args:
        model (Sequential): The neural network model to train
        train_data (Tuple[np.ndarray, np.ndarray]): Training data and labels
        val_data (Tuple[np.ndarray, np.ndarray]): Validation data and labels
        batch_size (int): Number of samples per gradient update (default: 32)
        epochs (int): Maximum number of training epochs (default: 20)

    Returns:
        Any: Training history containing loss and metric values
    """
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        min_delta=0.001
    )
    
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    history = model.fit(
        train_data[0],
        train_data[1],
        validation_data=val_data,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early_stopping],
        verbose=1
    )
    
    return history

def evaluate_model(model: Sequential, 
                  test_data: Tuple[np.ndarray, np.ndarray]) -> Tuple[float, float, float]:
    """
    Evaluates the trained model using multiple performance metrics.
    Provides a comprehensive evaluation of the model's performance on test data.

    Evaluation metrics:
    - Accuracy: Overall classification accuracy
    - Precision: Weighted precision across all classes
    - Recall: Weighted recall across all classes

    Args:
        model (Sequential): The trained neural network model
        test_data (Tuple[np.ndarray, np.ndarray]): Test data and ground truth labels

    Returns:
        Tuple[float, float, float]: Tuple containing accuracy, precision, and recall scores
    """
    X_test, y_test = test_data
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    accuracy = accuracy_score(y_test, y_pred_classes)
    precision = precision_score(y_test, y_pred_classes, average='weighted')
    recall = recall_score(y_test, y_pred_classes, average='weighted')
    
    return accuracy, precision, recall

# Glove embedding - 100


In [19]:
import gensim.downloader as api

In [20]:
def run_classification_pipeline_glove() -> None:
    """
    Runs the complete classification pipeline for GloVe embeddings with different dimensionalities (50, 100, 200).
    """
    print("=== Starting Author Classification Pipeline with GloVe ===")
    
    try:
        # Cargar embeddings preentrenados de GloVe
        glove_models = {
            #'glove_50': api.load('glove-wiki-gigaword-50'),   # 50 dimensiones
            #'glove_100': api.load('glove-wiki-gigaword-100'),  # 100 dimensiones
            'glove_200': api.load('glove-wiki-gigaword-200'),  # 200 dimensiones
        }

        print("\n1. Processing text data...")

        results = {}

        # Iterar sobre los modelos de GloVe de diferentes dimensiones
        for glove_name, glove_model in glove_models.items():
            embedding_size = int(glove_name.split('_')[1])  # Extraer el tamaño de los embeddings (50, 100, 200)
            
            print(f"\nPreparing embedding matrix and data for {glove_name}...")

            # Crear la matriz de embeddings para GloVe
            embedding_matrix = create_embedding_matrix(
                train_df, 
                val_df, 
                test_df, 
                glove_model,
                embedding_size
            )

            # Procesar los datos y convertir los tokens en índices usando el modelo GloVe
            X_train = prepare_data(train_df, processor_, word2vec_model=glove_model)
            X_val = prepare_data(val_df, processor_, word2vec_model=glove_model)
            X_test = prepare_data(test_df, processor_, word2vec_model=glove_model)

            # Obtener la longitud máxima de secuencia
            max_length = max(
                max(len(x) for x in X_train),
                max(len(x) for x in X_val),
                max(len(x) for x in X_test)
            )
            print(f"\nMaximum sequence length for {glove_name}: {max_length}")

            # Pad sequences
            print(f"\nPadding sequences for {glove_name}...")
            X_train_padded = pad_sequences(X_train, maxlen=max_length, padding='post')
            X_val_padded = pad_sequences(X_val, maxlen=max_length, padding='post')
            X_test_padded = pad_sequences(X_test, maxlen=max_length, padding='post')

            # Preparar etiquetas (solo una vez)
            print("\nEncoding labels...")
            label_encoder = LabelEncoder()
            label_encoder.fit(train_df['author'])
            y_train = label_encoder.transform(train_df['author'])
            y_val = label_encoder.transform(val_df['author'])
            y_test = label_encoder.transform(test_df['author'])

            # Obtener tamaño del vocabulario
            vocab_size = len(glove_model.key_to_index) + 1
            print(f"\nVocabulary size for {glove_name}: {vocab_size}")

            # Entrenar y evaluar los modelos
            models = {
                f'RDD1_{glove_name}': RDD1(embedding_matrix, vocab_size, embedding_size, max_length),
                f'RDD2_{glove_name}': RDD2(embedding_matrix, vocab_size, embedding_size, max_length),
                f'RDD3_{glove_name}': RDD3(embedding_matrix, vocab_size, embedding_size, max_length)
            }

            for model_name, model in models.items():
                print(f"\n=== Training {model_name} ===")
                print("\nModel Architecture:")
                model.summary()

                # Entrenar el modelo
                print(f"\nTraining model {model_name}...")
                history = train_model(
                    model=model,
                    train_data=(X_train_padded, y_train),
                    val_data=(X_val_padded, y_val),
                    batch_size=32,
                    epochs=20
                )

                # Evaluar y almacenar resultados
                accuracy, precision, recall = evaluate_model(
                    model=model,
                    test_data=(X_test_padded, y_test)
                )

                results[model_name] = {
                    'val_accuracy': max(history.history['val_accuracy']),
                    'train_accuracy': max(history.history['accuracy']),
                    'test_accuracy': accuracy,
                    'test_precision': precision,
                    'test_recall': recall
                }

                # Imprimir resultados
                print(f"\n{model_name} Results:")
                print(f"Best validation accuracy: {results[model_name]['val_accuracy']:.4f}")
                print(f"Best training accuracy: {results[model_name]['train_accuracy']:.4f}")
                print(f"Test Accuracy: {results[model_name]['test_accuracy']:.4f}")
                print(f"Test Precision: {results[model_name]['test_precision']:.4f}")
                print(f"Test Recall: {results[model_name]['test_recall']:.4f}")
                
                print("\n" + "="*50)

        # Comparación final de resultados
        print("\n=== Final Model Comparison ===")
        comparison_df = pd.DataFrame(results).round(4)
        print(comparison_df)
    
    except Exception as e:
        print(f"\nError in pipeline execution: {str(e)}")
        raise


In [21]:
run_classification_pipeline_glove()

=== Starting Author Classification Pipeline with GloVe ===
[==================================================] 100.0% 252.1/252.1MB downloaded

1. Processing text data...

Preparing embedding matrix and data for glove_200...
Procesando textos...
Procesando textos...
Procesando textos...
Longitud máxima de secuencia: 121
Glove model detected
Procesando textos...
Procesando textos...
Procesando textos...

Maximum sequence length for glove_200: 121

Padding sequences for glove_200...

Encoding labels...

Vocabulary size for glove_200: 400001


c:\Users\Pc\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



=== Training RDD1_glove_200 ===

Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │    80,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 121)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 121, 200)  │ 80,000,200 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d   │ (None, 121, 200)  │          0 │ embedding_2[0][0] │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 121, 512)  │    935,936 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 121, 512)  │      1,024 │ bidirectional[0]… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 121, 512)  │    525,568 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 121, 512)  │          0 │ layer_normalizat… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 121, 512)  │      1,024 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 121, 256)  │    656,384 │ layer_normalizat… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 121, 256)  │        512 │ bidirectional_1[… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_2     │ (None, 121, 128)  │    164,352 │ layer_normalizat… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 121, 128)  │        256 │ bidirectional_2[… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 896)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
│                     │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 82,912,715 (316.29 MB)

 Trainable params: 2,910,723 (11.10 MB)

 Non-trainable params: 80,001,992 (305.18 MB)


Training model RDD3_glove_200...
Epoch 1/20
333/333 ━━━━━━━━━━━━━━━━━━━━ 161s 453ms/step - accuracy: 0.4653 - loss: 1.2568 - val_accuracy: 0.7447 - val_loss: 0.8059
Epoch 2/20
333/333 ━━━━━━━━━━━━━━━━━━━━ 150s 450ms/step - accuracy: 0.7804 - loss: 0.5450 - val_accuracy: 0.8090 - val_loss: 0.4665
Epoch 3/20
333/333 ━━━━━━━━━━━━━━━━━━━━ 155s 465ms/step - accuracy: 0.8280 - loss: 0.4254 - val_accuracy: 0.8639 - val_loss: 0.3857
Epoch 4/20
333/333 ━━━━━━━━━━━━━━━━━━━━ 155s 464ms/step - accuracy: 0.8585 - loss: 0.3568 - val_accuracy: 0.8825 - val_loss: 0.2895
Epoch 5/20
333/333 ━━━━━━━━━━━━━━━━━━━━ 150s 450ms/step - accuracy: 0.8645 - loss: 0.3335 - val_accuracy: 0.8707 - val_loss: 0.3067
Epoch 6/20
333/333 ━━━━━━━━━━━━━━━━━━━━ 150s 451ms/step - accuracy: 0.8862 - loss: 0.2874 - val_accuracy: 0.9011 - val_loss: 0.2321
Epoch 7/20
333/333 ━━━━━━━━━━━━━━━━━━━━ 150s 450ms/step - accuracy: 0.8994 - loss: 0.2675 - val_accuracy: 0.9189 - val_loss: 0.2152
Epoch 8/20
333/333 ━━━━━━━━━━━━━━━━━━━━ 15